# 04 – Modeling & Evaluation
### Car Price Prediction Using Machine Learning
Group Assignment 02 - CCS3012 - Data Analytics  
Submission Date: 16th September 2025

---

### **Group 11**
-  **FC211034 - N.D. Samararathne Kodikara**
-  **FC211013 - N.W.V. Tharindu Pabasara**
-  **FC211025 - W.M.M.C.B. Wijesundara**



---


### **Supervisor**
**Ms. Dilmi Praveena**  
*Faculty of Computing*  
*University of Sri Jayewardenepura*

---


## 📌 Objectives

This notebook builds upon the cleaned dataset produced in **Notebook 01 — Data Exploration & Cleaning**.
The focus here is to prepare the dataset for modeling by performing outlier handling, feature engineering, and deterministic cleaning.

---

- Load processed train/test datasets and mappings from Notebook 02.

- Prepare numeric and categorical pipelines.

- Train and evaluate multiple regression models.

- Compare performance metrics (RMSE, MAE, R²) to select the best model.

- Save final model(s) for deployment or further analysis.


---

### 📂 Input  
- `clean_data.csv` saved in `Data/processed/`  
- `X_train.pkl, X_test.pkl`

- `y_train_log.pkl, y_test_log.pkl`

- `y_train_cap.pkl, y_test_cap.pkl`

- `categorical_mappings.pkl, model_mappings.pkl`


---

# Setup & imports

In [1]:
# Libraries
import pandas as pd
import numpy as np
import joblib
import pickle

# Sklearn imports (for pipelines and modeling later)
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score